In [9]:
import tensorflow as tf 
import tensorflow_datasets as tfds 

# Carga del dataset pcap_mnist
(pcap_data_train, pcap_data_test), pcap_ds_info = tfds.load( 
    'mnist', 
    split=['train', 'test'], 
    shuffle_files=True, 
    as_supervised=True, 
    with_info=True, 
) 

# Normalización de los datos
def pcap_normalize_data(pcap_image, pcap_label): 
    # Convertir a float y asegurar forma (28, 28, 1)
    pcap_image = tf.cast(pcap_image, tf.float32) / 255.
    pcap_image = tf.reshape(pcap_image, [28, 28, 1])
    return pcap_image, pcap_label
 

# Normaliza el dataset de ENTRENAMIENTO
pcap_data_train = pcap_data_train.map(
    pcap_normalize_data, 
    num_parallel_calls=tf.data.AUTOTUNE
)

# Normaliza el dataset de PRUEBA
pcap_data_test = pcap_data_test.map(
    pcap_normalize_data,  
    num_parallel_calls=tf.data.AUTOTUNE
) 

BATCH_SIZE = 32

pcap_data_train = pcap_data_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
pcap_data_test = pcap_data_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


# Definición del modelo
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

# Compilación del modelo  
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
) 

# Entrenamiento del modelo
model.fit(pcap_data_train, epochs=6) 

# Predicción
model.predict(pcap_data_test)

Epoch 1/6
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2565 - sparse_categorical_accuracy: 0.9269
Epoch 2/6
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1152 - sparse_categorical_accuracy: 0.9654
Epoch 3/6
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0784 - sparse_categorical_accuracy: 0.9763
Epoch 4/6
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0573 - sparse_categorical_accuracy: 0.9827
Epoch 5/6
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0423 - sparse_categorical_accuracy: 0.9876
Epoch 6/6
313/313 [==============================] - 1s 2ms/step


array([[ -5.913885 ,  -8.508826 ,  11.58493  , ..., -11.980821 ,
         -2.5803628,  -5.1940904],
       [ 18.75349  , -17.920616 ,  -9.192937 , ...,  -8.552802 ,
         -7.071189 , -10.806072 ],
       [-11.802656 , -10.261378 ,  -6.8189917, ...,  -1.1689671,
         -3.1073961,   4.1730533],
       ...,
       [ -8.2736025,  -9.40512  ,   1.876831 , ..., -13.161524 ,
         10.171167 ,  -3.6289816],
       [ 14.021061 , -10.94169  ,   1.8975983, ..., -11.173404 ,
        -12.240341 ,  -1.0961963],
       [-12.517393 ,  -7.057191 , -10.277754 , ..., -14.159073 ,
         -5.4150295,  -4.987723 ]], dtype=float32)

**Un ejemplo de análisis visual del tráfico de red**

 Preprocesamiento del conjunto de datos.

Las líneas 2 a 3 abren los archivos de flujo de red divididos mediante el modo rb y el contenido leído se convierte a formato hexadecimal mediante la función binascii.hexlify(). El contenido hexadecimal actual se rellenará y recortará a 784 bits en las líneas 4 a 7. Las líneas 8 a 9 almacenan el contenido hexadecimal convertido para su posterior uso

In [ ]:
# MNIST_Preparing_session_png.py 
def getMatrixfrom_pcap(filename,width): 
with open (filename, 'rb') as f: 
      content = f.read() 
      hexst = binascii.hexlify(content) 
      fh = numpy.array([int (hexst[i: i+2],16) 
             for i in range (0, len(hexst), 2)]) 
      rn = len(fh)/width 
      fh = numpy.reshape(fh[:rn*width], (-1, width)) 
      fh = numpy. uint8(fh) 
      return fh 

La línea 1 marca el inicio de una iteración, que se utiliza para gestionar todas las imágenes convertidas del flujo de red. Las líneas 2 y 3 definen matrices para almacenar la imagen y su etiqueta correspondiente. Las líneas 5 a 10 buscan los archivos que terminan en .png y los 
añaden a una lista. La línea 12 reorganiza los archivos en la lista y prepara el proceso de división del conjunto de datos. Este fragmento de código se encuentra en el siguiente enlace: 
https://github.com/PegX/Falcon/blob/master/4_Png2Mnist.py

In [5]:
import tensorflow_datasets as tfds
import tensorflow as tf
from PIL import Image
import os

# Cargar el dataset MNIST
(ds_train, _), _ = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)

# Crear carpetas destino como 4_Png/Train/0, ..., 4_Png/Train/9
base_dir = "4_Png/Train"
for i in range(10):
    os.makedirs(os.path.join(base_dir, str(i)), exist_ok=True)

# Guardar imágenes como .png en sus carpetas respectivas
count = 0
for image, label in tfds.as_numpy(ds_train):
    label_str = str(label)
    img = Image.fromarray(image.squeeze(), mode='L')
    img.save(os.path.join(base_dir, label_str, f"{count}.png"))
    count += 1
    if count >= 10:  # Limita a 10 imágenes
        break


In [6]:
import os
from PIL import Image
from array import array
from random import shuffle

Names = [['4_Png\Train','5_Mnist\\Train']]

for name in Names:
    data_image = array('B')
    data_label = array('B')

    FileList = []
    for dirname in os.listdir(name[0]):
        path = os.path.join(name[0], dirname)
        for filename in os.listdir(path):
            if filename.endswith(".png"):
                FileList.append(os.path.join(name[0], dirname, filename))

    shuffle(FileList)

    for filename in FileList:
        label = int(filename.split('\\')[2])
        Im = Image.open(filename)
        pixel = Im.load()
        width, height = Im.size
        for x in range(width):
            for y in range(height):
                data_image.append(pixel[y, x])
        data_label.append(label)  # labels start (one unsigned byte each)

    hexval = "{0:#0{1}x}".format(len(FileList), 6)  # number of files in HEX
    hexval = '0x' + hexval[2:].zfill(8)

    header = array('B')
    header.extend([0, 0, 8, 1])
    header.append(int('0x' + hexval[2:][0:2], 16))
    header.append(int('0x' + hexval[2:][2:4], 16))
    header.append(int('0x' + hexval[2:][4:6], 16))
    header.append(int('0x' + hexval[2:][6:8], 16))

    data_label = header + data_label

    if max([width, height]) <= 256:
        header.extend([0, 0, 0, width, 0, 0, 0, height])
    else:
        raise ValueError('Image exceeds maximum size: 256x256 pixels')

    header[3] = 3  # Changing MSB for image data (0x00000803)
    data_image = header + data_image

    with open(name[1] + '-images-idx3-ubyte', 'wb') as output_file:
        data_image.tofile(output_file)

    with open(name[1] + '-labels-idx1-ubyte', 'wb') as output_file:
        data_label.tofile(output_file)

Las líneas 14 a 22 presentan el proceso de lectura de la imagen y su etiqueta correspondiente en la memoria mediante la biblioteca Pillow. En nuestro caso, utilizamos las funciones Image.open() y load() en el atributo de tamaño de la imagen para almacenar las imágenes e incorporarlas al conjunto de datos.
Las líneas 25 a 45 almacenan todas las imágenes y etiquetas leídas en cuatro archivos. Hay 
matrices de imágenes para entrenamiento y prueba, así como las etiquetas de entrenamiento 
y prueba. La segunda parte es el entrenamiento del modelo. Con el conjunto de datos de 
entrenamiento dividido y la red neuronal definida, entrenamos los modelos de IA con 
múltiples rondas. La última parte es la prueba o predicción del modelo. 

In [7]:
tf.keras.models.Sequential( 
[tf.keras.layers.Flatten(input_shape= (28,28)), 
tf.keras.layers.Dense(128, activation='relu'), 
tf.keras.layers.Dense(10)] 
)

In [16]:
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [17]:
#compilar el modelo y declarar la función de pérdida y las métricas de rendimiento para el entrenamiento
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])

model.compile( 
optimizer=tf.keras.optimizers.Adam(0.001), 
loss=tf.keras.losses.SparseCategoricalCrossentropy( 
from_logits=True), 
metrics=[tf.keras.metrics.SparseCategoricalAccuracy()], 
) 

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 5s 5ms/step - loss: 0.3550 - sparse_categorical_accuracy: 0.9018 - val_loss: 0.1857 - val_sparse_categorical_accuracy: 0.9470
Epoch 2/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1598 - sparse_categorical_accuracy: 0.9537 - val_loss: 0.1324 - val_sparse_categorical_accuracy: 0.9631
Epoch 3/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1135 - sparse_categorical_accuracy: 0.9673 - val_loss: 0.1058 - val_sparse_categorical_accuracy: 0.9692
Epoch 4/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0885 - sparse_categorical_accuracy: 0.9746 - val_loss: 0.0933 - val_sparse_categorical_accuracy: 0.9725
Epoch 5/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0717 - sparse_categorical_accuracy: 0.9791 - val_loss: 0.0834 - val_sparse_categorical_accuracy: 0.9743
Epoch 6/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0593 - sparse_categorical_accu